In [23]:
prompt = "A model that takes in a puzzle-like reasoning-heavy question in English, and responds with a well-reasoned, step-by-step thought out response in Spanish."
temperature = .4
number_of_examples = 50

# Load required packages to run this code

In [4]:
!pip install openai tenacity python-dotenv


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# Load variables using dot_env

In [2]:
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

True

# Load os, openai, random, tenacity and set openai apikey

In [3]:
import os
import openai
import random
from tenacity import retry, stop_after_attempt, wait_exponential
openai.api_key = os.getenv("OPENAI_APIKEY")

# Gererate random training data

In [9]:
N_RETRIES = 3

@retry(stop=stop_after_attempt(N_RETRIES), wait=wait_exponential(multiplier=1, min=4, max=70))
def generate_example(prompt, prev_examples, temperature=.5):
    messages=[
        {
            "role": "system",
            "content": f"You are generating data which will be used to train a machine learning model.\n\nYou will be given a high-level description of the model we want to train, and from that, you will generate data samples, each with a prompt/response pair.\n\nYou will do so in this format:\n```\nprompt\n-----------\n$prompt_goes_here\n-----------\n\nresponse\n-----------\n$response_goes_here\n-----------\n```\n\nOnly one prompt/response pair should be generated per turn.\n\nFor each turn, make the example slightly more complex than the last, while ensuring diversity.\n\nMake sure your samples are unique and diverse, yet high-quality and complex enough to train a well-performing model.\n\nHere is the type of model we want to train:\n`{prompt}`"
        }
    ]

    if len(prev_examples) > 0:
        if len(prev_examples) > 8:
            prev_examples = random.sample(prev_examples, 8)
        for example in prev_examples:
            messages.append({
                "role": "assistant",
                "content": example
            })

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=messages,
        temperature=temperature,
        max_tokens=1000,
    )

    return response.choices[0].message['content']

# Generate examples
prev_examples = []
for i in range(number_of_examples):
    print(f'Generating example {i}')
    example = generate_example(prompt, prev_examples, temperature)
    prev_examples.append(example)

print(prev_examples)

Generating example 0
Generating example 1
Generating example 2
Generating example 3
Generating example 4
Generating example 5
Generating example 6
Generating example 7
Generating example 8
Generating example 9
Generating example 10
Generating example 11
Generating example 12
Generating example 13


KeyboardInterrupt: 

# Define system message

In [12]:
def generate_system_message(prompt):

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
          {
            "role": "system",
            "content": "You will be given a high-level description of the model we are training, and from that, you will generate a simple system prompt for that model to use. Remember, you are not generating the system message for data generation -- you are generating the system message to use for inference. A good format to follow is `Given $INPUT_DATA, you will $WHAT_THE_MODEL_SHOULD_DO.`.\n\nMake it as concise as possible. Include nothing but the system prompt in your response.\n\nFor example, never write: `\"$SYSTEM_PROMPT_HERE\"`.\n\nIt should be like: `$SYSTEM_PROMPT_HERE`."
          },
          {
              "role": "user",
              "content": prompt.strip(),
          }
        ],
        temperature=temperature,
        max_tokens=500,
    )

    return response.choices[0].message['content']

system_message = generate_system_message(prompt)

print(f'The system message is: `{system_message}`. Feel free to re-run this cell if you want a better result.')

The system message is: `Given a puzzle-like reasoning-heavy question in English, you will provide a well-reasoned, step-by-step thought out response in Spanish.`. Feel free to re-run this cell if you want a better result.


# Create training_examples.jsonl based on examples

In [14]:
import json
import pandas as pd
import os

# Initialize lists to store prompts and responses
prompts = []
responses = []

# Parse out prompts and responses from examples
for example in prev_examples:
    try:
        split_example = example.split('-----------')
        prompts.append(split_example[1].strip())
        responses.append(split_example[3].strip())
    except:
        pass

# Create a DataFrame
df = pd.DataFrame({
    'prompt': prompts,
    'response': responses
})

# Remove duplicates
df = df.drop_duplicates()

print('There are ' + str(len(df)) + ' successfully-generated examples.')

# Define the system message
system_message = "You are generating data which will be used to train a machine learning model."

# Initialize list to store training examples
training_examples = []

# Create training examples in the format required for GPT-3.5 fine-tuning
for index, row in df.iterrows():
    training_example = {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": row['prompt']},
            {"role": "assistant", "content": row['response']}
        ]
    }
    training_examples.append(training_example)

# Define the filename for the JSON file
json_filename = 'training_examples.jsonl'

# Check if the JSON file already exists, and if not, create it
if not os.path.exists(json_filename):
    with open(json_filename, 'w') as f:
        for example in training_examples:
            f.write(json.dumps(example) + '\n')
    print(f'Created {json_filename} with {len(training_examples)} examples.')
else:
    print(f'{json_filename} already exists. Not overwriting it.')

# Now, your training examples are saved to 'training_examples.jsonl' if it didn't already exist.


There are 13 successfully-generated examples.
training_examples.jsonl already exists. Not overwriting it.


# open the file

In [41]:
file_id = openai.File.create(
  file=open("training_examples.jsonl", "rb"),
  purpose='fine-tune'
).id

In [42]:
print(file_id)

file-9r1QxFI2M7VfgwJyVKtQRtIn


# Create the fine tuning job

In [15]:
job = openai.FineTuningJob.create(training_file=file_id, model="gpt-3.5-turbo")

job_id = job.id

In [56]:
job_id = job.id
print(job_id)

ftjob-BBsjv9n7gi7MF31UC2RKLmvh


# List fine tuning events

In [58]:
response = openai.FineTuningJob.list_events(id=job_id, limit=50)

events = response['data']
events.reverse()

for event in events:
    print(event["message"])

Step 44/91: training loss=0.03
Step 45/91: training loss=0.13
Step 46/91: training loss=0.18
Step 47/91: training loss=0.06
Step 48/91: training loss=0.03
Step 49/91: training loss=0.03
Step 50/91: training loss=0.01
Step 51/91: training loss=0.01
Step 52/91: training loss=0.14
Step 53/91: training loss=0.02
Step 54/91: training loss=0.04
Step 55/91: training loss=0.09
Step 56/91: training loss=0.00
Step 57/91: training loss=0.11
Step 58/91: training loss=0.02
Step 59/91: training loss=0.02
Step 60/91: training loss=0.02
Step 61/91: training loss=0.01
Step 62/91: training loss=0.00
Step 63/91: training loss=0.01
Step 64/91: training loss=0.12
Step 65/91: training loss=0.00
Step 66/91: training loss=0.01
Step 67/91: training loss=0.02
Step 68/91: training loss=0.01
Step 69/91: training loss=0.01
Step 70/91: training loss=0.02
Step 71/91: training loss=0.00
Step 72/91: training loss=0.01
Step 73/91: training loss=0.06
Step 74/91: training loss=0.00
Step 75/91: training loss=0.05
Step 76/